# Load SSP Grid

In [1]:
# NBVAL_SKIP
from rubix.spectra.ssp.templates import BruzualCharlot2003

print(BruzualCharlot2003)


2024-06-05 21:18:05,125 - rubix - INFO - [SSPModels] File /Users/buck/Documents/Nexus/projects/Rubix/rubix/spectra/ssp/templates/MaStar_CB19.slog_1_5.fits not found. Downloading it from http://ifs.astroscu.unam.mx/pyPipe3D/templates/
2024-06-05 21:18:05,460 - rubix - ERROR - [SSPModels] Error: 404 Client Error: Not Found for url: http://ifs.astroscu.unam.mx/pyPipe3D/templates/MaStar_CB19.slog_1_5.fits


FileNotFoundError: Could not download file MaStar_CB19.slog_1_5.fits from url http://ifs.astroscu.unam.mx/pyPipe3D/templates/.

SSP lookup

In [2]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
from rubix.spectra.ssp.templates import BruzualCharlot2003
from jax import jit

ssp = BruzualCharlot2003
wave = ssp.wavelength


age_index = 0
met_index = 3

target_age = ssp.age[age_index] + 0.5*(ssp.age[age_index+1] - ssp.age[age_index])
print(target_age)
target_met = ssp.metallicity[met_index] + 0.5*(ssp.metallicity[met_index+1] - ssp.metallicity[met_index])

lookup = ssp.get_lookup_interpolation()

spec_calc = lookup(target_met, target_age)

spec_true = ssp.flux[met_index, age_index, :]

plt.plot(wave, spec_calc, label='calc')
plt.plot(wave, spec_true, label='true')

plt.legend()
plt.yscale('log')

2024-06-05 21:18:11,891 - rubix - INFO - [SSPModels] File /Users/buck/Documents/Nexus/projects/Rubix/rubix/spectra/ssp/templates/MaStar_CB19.slog_1_5.fits not found. Downloading it from http://ifs.astroscu.unam.mx/pyPipe3D/templates/
2024-06-05 21:18:12,226 - rubix - ERROR - [SSPModels] Error: 404 Client Error: Not Found for url: http://ifs.astroscu.unam.mx/pyPipe3D/templates/MaStar_CB19.slog_1_5.fits


FileNotFoundError: Could not download file MaStar_CB19.slog_1_5.fits from url http://ifs.astroscu.unam.mx/pyPipe3D/templates/.

In [3]:
# NBVAL_SKIP
# Check if it works with jit

spec_calc = jit(lookup)(target_met, target_age)

plt.plot(wave, spec_calc, label='calc jit')

NameError: name 'jit' is not defined

In [ ]:
# NBVAL_SKIP
from rubix.utils import load_galaxy_data

data, units = load_galaxy_data("output/rubix_galaxy.h5")
mass = data["particle_data"]["stars"]["mass"]
metallicity = data["particle_data"]["stars"]["metallicity"]
age = data["particle_data"]["stars"]["age"]

Vmap

In [ ]:
# NBVAL_SKIP
# Calculate spectra with vmap
from jax import vmap

lookup = ssp.get_lookup_interpolation()

subset = 1000

# Use only subset because it is too big to fit into gpu memory
met_subset = metallicity[:subset]
age_subset = age[:subset]


# Clip the metallicity and age values to the range of the SSP

met_subset = met_subset.clip(min(ssp.metallicity), max(ssp.metallicity))
age_subset = age_subset.clip(min(ssp.age), max(ssp.age))


spec_calc = vmap(lookup)(met_subset, age_subset)


spec_calc.shape


In [ ]:
# NBVAL_SKIP
# check if it contains nan values
import jax.numpy as jnp
jnp.isnan(spec_calc).any()

# Use configuration to load lookup function

In [ ]:
config ={ "ssp": {
    "template": {
      "name": "BruzualCharlot2003"
    },
    "method": "cubic"
  }
}

In [ ]:
# NBVAL_SKIP
from rubix.core.ssp import get_lookup_interpolation

lookup = get_lookup_interpolation(config)

In [ ]:
# NBVAL_SKIP
# Check how many particles are outside the range of the SSP
import numpy as np
np.sum(metallicity < ssp.metallicity[0]), np.sum(metallicity > ssp.metallicity[-1])

In [ ]:
# NBVAL_SKIP
np.sum(age < ssp.age[0]), np.sum(age > ssp.age[-1])

In [ ]:
# NBVAL_SKIP
len(metallicity)

In [ ]:
# NBVAL_SKIP
# clip the metallicity and age values to the range of the SSP
met_subset = met_subset.clip(min(ssp.metallicity), max(ssp.metallicity))
age_subset = age_subset.clip(min(ssp.age), max(ssp.age))
lookup(met_subset, age_subset)